In [1]:
import sys 
from pyspark.sql import SparkSession 
from pyspark.sql.functions import count
from pyspark.sql.functions import col

ModuleNotFoundError: No module named 'pyspark'

In [2]:
# Build a SparkSession using the SparkSession APIs.
# If one does not exist, then create an instance. There # can only be one SparkSession per JVM.
spark = (SparkSession
         .builder
         .appName("FocusFileApp")
         .getOrCreate())

# Get the focus file path
focus_file = "/Users/tonkata/Documents/Careem/focus-one-report-00001.csv"

# Read the file into a Spark DataFrame using the CSV
# format by inferring the schema and specifying that the
# file contains a header, which provides column names for comma- # separated fields.
focus_df = (spark.read.format("csv")
     .option("header", "true")
     .option("inferSchema", "true")
     .load(focus_file))

25/01/02 19:02:32 WARN Utils: Your hostname, antonov-mac-dev.local resolves to a loopback address: 127.0.0.1; using 192.168.18.243 instead (on interface en0)
25/01/02 19:02:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/02 19:02:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

In [4]:
# We use the DataFrame high-level APIs. Note
# that we don't use RDDs at all. Because some of Spark's
# functions return the same object, we can chain function calls.
# 1. Select from the DataFrame the fields "State", "Color", and "Count" 
# 2. Since we want to group each state and its M&M color count,
# we use groupBy()
# 3. Aggregate counts of all colors and groupBy() State and Color
# 4 orderBy() in descending order
count_focus_df = (focus_df
                  .filter((col("CommitmentDiscountType").isNotNull()) & (col("CommitmentDiscountType") != ""))
                  .select("CommitmentDiscountType", "ConsumedUnit", "ConsumedQuantity")
                  .groupBy("CommitmentDiscountType", "ConsumedUnit")
                  .agg(count("ConsumedQuantity").alias("Total"))
                  .orderBy("Total", ascending=False))

# Show the resulting aggregations for all the states and colors;
# a total count of each color per state.
# Note show() is an action, which will trigger the above
# query to be executed.
count_focus_df.show(n=60, truncate=False) 
print("Total Rows = %d" % (count_focus_df.count()))

+----------------------+------------+-----+
|CommitmentDiscountType|ConsumedUnit|Total|
+----------------------+------------+-----+
|Savings Plan          |Hours       |5234 |
|Savings Plan          |Requests    |65   |
|Savings Plan          |Seconds     |46   |
+----------------------+------------+-----+

Total Rows = 3


In [5]:
from pyspark.sql import Row
blog_row = Row(6, "Reynold", "Xin", "https://tinyurl.6", 255568, "3/2/2015",["twitter", "LinkedIn"])
# access using index for individual items 
blog_row[1]


'Reynold'

In [7]:
rows = [Row("Matei Zaharia", "CA"), Row("Reynold Xin", "CA")]
authors_df = spark.createDataFrame(rows, ["Authors", "State"])
authors_df.show()

25/01/02 19:03:29 ERROR Executor: Exception in task 0.0 in stage 21.0 (TID 63)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/tonkata/Library/Python/3.9/lib/python/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1100, in main
    raise PySparkRuntimeError(
    ...<5 lines>...
    )
pyspark.errors.exceptions.base.PySparkRuntimeError: [PYTHON_VERSION_MISMATCH] Python in worker has different version (3, 13) than that in driver 3.9, PySpark cannot run with different minor versions.
Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:784)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:766)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(Pytho

Py4JJavaError: An error occurred while calling o152.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 21.0 failed 1 times, most recent failure: Lost task 0.0 in stage 21.0 (TID 63) (192.168.18.243 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/tonkata/Library/Python/3.9/lib/python/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1100, in main
    raise PySparkRuntimeError(
    ...<5 lines>...
    )
pyspark.errors.exceptions.base.PySparkRuntimeError: [PYTHON_VERSION_MISMATCH] Python in worker has different version (3, 13) than that in driver 3.9, PySpark cannot run with different minor versions.
Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(